In [6]:
# Load all the important libraries to create the KNN Model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [7]:
#import the data set

Data = pd.read_csv(r"C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Dataset\CKD.csv")

In [8]:
Data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3.0,0.0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1.0,0.0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.0,76.459948,d,1.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.0,50.000000,c,0.0,0.0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes


In [9]:
#Lets check total number of rows and columns 
Data.shape

(399, 25)

In [10]:
#Check the dataset info 
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             399 non-null    float64
 1   bp              399 non-null    float64
 2   sg              399 non-null    object 
 3   al              399 non-null    float64
 4   su              399 non-null    float64
 5   rbc             399 non-null    object 
 6   pc              399 non-null    object 
 7   pcc             399 non-null    object 
 8   ba              399 non-null    object 
 9   bgr             399 non-null    float64
 10  bu              399 non-null    float64
 11  sc              399 non-null    float64
 12  sod             399 non-null    float64
 13  pot             399 non-null    float64
 14  hrmo            399 non-null    float64
 15  pcv             399 non-null    float64
 16  wc              399 non-null    float64
 17  rc              399 non-null    flo

In [11]:
# Lets check the description of data set
Data.describe()

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,pcv,wc,rc
count,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000,399.000000
mean,51.492308,76.459948,0.899749,0.395990,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,38.868902,8408.191126,4.705597
std,16.995379,13.492053,1.314769,1.041155,74.864224,49.336046,5.623758,9.215829,2.823323,2.715753,8.157274,2526.204544,0.841006
min,2.000000,50.000000,0.000000,0.000000,22.000000,1.500000,0.400000,4.500000,2.500000,3.100000,9.000000,2200.000000,2.100000
25%,42.000000,70.000000,0.000000,0.000000,101.000000,27.000000,0.900000,135.000000,4.000000,10.850000,34.000000,6950.000000,4.500000
50%,54.000000,76.459948,0.000000,0.000000,127.000000,44.000000,1.400000,137.528754,4.627244,12.518156,38.868902,8408.191126,4.705597
75%,64.000000,80.000000,2.000000,0.000000,150.000000,62.500000,3.077356,141.000000,4.800000,14.600000,44.000000,9400.000000,5.100000
max,90.000000,180.000000,5.000000,5.000000,490.000000,391.000000,76.000000,163.000000,47.000000,17.800000,54.000000,26400.000000,8.000000


In [12]:
# Lets check if the data set has NA values
Data[Data.isna().any(axis=1)]

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification


In [13]:
# Lets check if the data set has Null values
Data[Data.isnull().any(axis=1)]

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification


In [14]:
# Lets check how many classes are there in the target variable "Classification"
Data.classification.value_counts()

yes    249
no     150
Name: classification, dtype: int64

# From above output we got to know that there are 250 records in "Yes" class and 150 records in "No" class. so its called as imbalanced data set

In [15]:
# Lets upsample the "No" records to match with "Yes" records
Data_No = Data[Data["classification"]=="no"]

In [16]:
import random

samples_index = random.sample(range(150),99)

In [17]:
Data_No_final = Data_No.iloc[samples_index]

In [18]:
Data_No_final.reset_index(inplace=True,drop=True)

In [19]:
Data_No_final

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,73.0,60.0,a,0.0,0.0,normal,normal,notpresent,notpresent,127.0,...,52.0,11000.0,4.7,no,no,no,yes,poor,no,no
1,38.0,80.0,a,0.0,0.0,normal,normal,notpresent,notpresent,99.0,...,44.0,7300.0,6.4,no,no,no,yes,poor,no,no
2,41.0,80.0,a,0.0,0.0,normal,normal,notpresent,notpresent,122.0,...,41.0,9100.0,5.2,no,no,no,yes,poor,no,no
3,25.0,60.0,a,0.0,0.0,normal,normal,notpresent,notpresent,119.0,...,40.0,9200.0,5.2,no,no,no,yes,poor,no,no
4,41.0,80.0,b,0.0,0.0,normal,normal,notpresent,notpresent,112.0,...,52.0,7200.0,5.8,no,no,no,yes,poor,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,34.0,70.0,b,0.0,0.0,normal,normal,notpresent,notpresent,87.0,...,47.0,7400.0,6.1,no,no,no,yes,poor,no,no
95,50.0,80.0,a,0.0,0.0,normal,normal,notpresent,notpresent,137.0,...,45.0,9500.0,4.6,no,no,no,yes,poor,no,no
96,43.0,60.0,b,0.0,0.0,normal,normal,notpresent,notpresent,108.0,...,43.0,7200.0,5.5,no,no,no,yes,poor,no,no
97,69.0,70.0,a,0.0,0.0,normal,normal,notpresent,notpresent,83.0,...,50.0,9300.0,5.4,no,no,no,yes,poor,no,no


In [20]:
Data = Data.append(Data_No_final,ignore_index=True)

C:\Users\Vinoth\AppData\Local\Temp\ipykernel_22684\3974467208.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Data = Data.append(Data_No_final,ignore_index=True)


In [21]:
Data.classification.value_counts()

yes    249
no     249
Name: classification, dtype: int64

# From above output we have upsampled the "No" class counts to match with "Yes" counts

In [22]:
# Lets convert the nominal categoty columns to numerical columns using one-hot-encoding via pandas get_dummies method

Data = pd.get_dummies(Data,drop_first=True)

In [23]:
Data.head()

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.0,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.0,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.0,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.0,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.0,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1


In [24]:
Data.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [25]:
# Lets seperate the dependent and independent variables
dependent = Data[['classification_yes']]
independent = Data[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]

# Lets scale all the columns to same measuremet scale from 0 to 1 using standardisation as you can see everu column values are in different measurements

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
independent = scaler.fit_transform(independent)

In [27]:
import sklearn
sklearn.metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_

In [28]:
# Lets spli the data to trai  and test set

X_Train,X_Test,Y_Train,Y_Test = train_test_split(independent,dependent,random_state=0,test_size=30)

In [29]:
# Model Parameters 
Model_Params = {"n_neighbors":[i for i in range(0,20) if i%2!=0 ],'weights':['uniform', 'distance'],"algorithm":['auto', 'ball_tree', 'kd_tree', 'brute'],
               "leaf_size":[i for i in range(30,80,5)],'p':[1,2],'metric':['cityblock','cosine','euclidean','haversine'],
               'n_jobs':[-1]}

In [31]:
# Lets create a model now using the Gridsearch CV

KNN_Model_Creation = GridSearchCV(KNeighborsClassifier(),Model_Params,n_jobs=-1,scoring={"roc_auc_score":'roc_auc','f1_weighted_score':'f1_weighted'},
                                            refit='roc_auc_score',verbose=2)

In [32]:
KNN_Model_Creation.fit(X_Train,Y_Train)

Fitting 5 folds for each of 6400 candidates, totalling 32000 fits


C:\Users\Vinoth\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
8000 fits failed out of a total of 32000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2000 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Vinoth\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Vinoth\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neighbors\_classification.py", line 200, in fit
    return self._fit(X, y)
  File "C:\Users\Vinoth\AppData\Local\Programs\Python\Python310\lib\site-pac

GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [30, 35, 40, 45, 50, 55, 60, 65, 70, 75],
                         'metric': ['cityblock', 'cosine', 'euclidean',
                                    'haversine'],
                         'n_jobs': [-1],
                         'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             refit='roc_auc_score',
             scoring={'f1_weighted_score': 'f1_weighted',
                      'roc_auc_score': 'roc_auc'},
             verbose=2)

In [33]:
# The best parameter that has givem high performance scores
KNN_Model_Creation.best_params_

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'cosine',
 'n_jobs': -1,
 'n_neighbors': 5,
 'p': 1,
 'weights': 'distance'}

In [35]:
# The best score for the above parameters
KNN_Model_Creation.best_score_

1.0

# Since there are lot of failures in the gridsearch CV results so the entries will be NA. so we don't need those entries to be saved as it may confuse us in future so we are removing those NA entries and have only the valid entries of grid search CV results saved and used for future reference

In [36]:
# Creating as data frame to save the GridSearch CV results
CV_Output = pd.DataFrame.from_dict(KNN_Model_Creation.cv_results_)

In [37]:
# Check the entries count
CV_Output.shape

(6400, 28)

In [38]:
# Get the indexes of all the records that has NA entries in the dataframe
index_of_NA_Vals = CV_Output[CV_Output.isna().any(axis=1)].index

In [39]:
index_of_NA_Vals

Int64Index([ 120,  121,  122,  123,  124,  125,  126,  127,  128,  129,
            ...
            6390, 6391, 6392, 6393, 6394, 6395, 6396, 6397, 6398, 6399],
           dtype='int64', length=2400)

In [40]:
# We are dropping all the indexed records that has NA entries in it
CV_Output.drop(index_of_NA_Vals,axis=0,inplace=True)

In [41]:
# We are resetting the index from 0 to 'n' post removing the above indexed records 
CV_Output.reset_index(drop=True,inplace=True)

In [54]:
# Final Data frame that has valid entries in it
CV_Output

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_leaf_size,param_metric,param_n_jobs,param_n_neighbors,param_p,...,std_test_roc_auc_score,rank_test_roc_auc_score,split0_test_f1_weighted_score,split1_test_f1_weighted_score,split2_test_f1_weighted_score,split3_test_f1_weighted_score,split4_test_f1_weighted_score,mean_test_f1_weighted_score,std_test_f1_weighted_score,rank_test_f1_weighted_score
0,0.004800,0.003919,0.221182,0.019352,auto,30,cityblock,-1,1,1,...,0.020185,3841,1.000000,0.978714,0.968053,0.946112,1.00000,0.978576,0.020409,521
1,0.007506,0.007872,0.217611,0.025822,auto,30,cityblock,-1,1,1,...,0.020185,3841,1.000000,0.978714,0.968053,0.946112,1.00000,0.978576,0.020409,521
2,0.006401,0.003200,0.235475,0.009596,auto,30,cityblock,-1,1,2,...,0.020185,3841,1.000000,0.978714,0.968053,0.946112,1.00000,0.978576,0.020409,521
3,0.006404,0.003202,0.225764,0.010689,auto,30,cityblock,-1,1,2,...,0.020185,3841,1.000000,0.978714,0.968053,0.946112,1.00000,0.978576,0.020409,521
4,0.000000,0.000000,0.011200,0.003920,auto,30,cityblock,-1,3,1,...,0.016406,3761,0.989360,0.957370,0.968053,0.935259,1.00000,0.970008,0.023000,1401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,0.004902,0.001741,0.011487,0.002393,brute,75,euclidean,-1,17,2,...,0.004346,841,0.968053,0.946658,0.935909,0.902414,0.96772,0.944151,0.024261,3441
3996,0.004202,0.000747,0.016803,0.005673,brute,75,euclidean,-1,19,1,...,0.004390,2081,0.946658,0.914273,0.903370,0.868971,0.96772,0.920198,0.034354,3921
3997,0.004507,0.000889,0.012248,0.005181,brute,75,euclidean,-1,19,1,...,0.004436,1281,0.968053,0.925117,0.914273,0.891344,0.96772,0.933301,0.030271,3761
3998,0.005198,0.002858,0.016389,0.004610,brute,75,euclidean,-1,19,2,...,0.004390,2081,0.946658,0.914273,0.903370,0.868971,0.96772,0.920198,0.034354,3921


In [55]:
# Lets save the grid search CV results to the csv file
CV_Output.to_csv(r"C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Grid Serach CV Results\KNN_GridSerachCV_Results.csv")

In [56]:
# Lets test the model with best parameter that we have got agains the test data and check the performance

Y_Predicted = KNN_Model_Creation.predict(X_Test)

In [57]:
Y_Predicted

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1], dtype=uint8)

In [58]:
# Lets check the confusion matrix for the above predicted against the actual results
confusion_matrix(Y_Predicted,Y_Test)

array([[16,  0],
       [ 0, 14]], dtype=int64)

In [59]:
#lets check the classification test report for the predicted against the actual results
print(classification_report(Y_Predicted,Y_Test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        14

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [60]:
KNN_Model_Creation.predict_proba(X_Test)[:,1]

array([0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.])

In [61]:
# lets check the roc_auc_score results for the predicted against the actual results
roc_auc_score(Y_Test,KNN_Model_Creation.predict_proba(X_Test)[:,1])

1.0

In [62]:
# Wow we can see our auc score is 100 percent and weighted f1 score is also 93 percent lets save out model
import pickle
pickle.dump(KNN_Model_Creation,open(r'C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Final Model\KNN_Final_Model.sav','wb'))

In [51]:
# Lets load and test our model
def Final_model_prod(model,columns,stdscaler='None'):
    query_values=[]
    for col_idx in range(0,len(columns)-1):
        query_values.append(float(input("Please enter valid {}: \n Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' ".format(columns[col_idx]))))
    pred_class=model.predict(stdscaler.transform([query_values]))
    
    if pred_class==0:
        print ("This patient doesn't have Chronic Kidney Disorder")
    else:
        print ("This patient have Chronic Kidney Disorder Please proceed proper medication")

In [52]:
# Load the saved model
Final_model= pickle.load(open(r"C:\Users\Vinoth\Desktop\HOPE AI\Machine Learning\Classification_Models\Chronic Kidney Disease Prediction Assignment\Final Model\KNN_Final_Model.sav",'rb'))

# here the scaler is the varaible that we have used for standardising the independent variables so we should use the same variable here  


In [229]:
# Test the final model
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore", category=UserWarning)
Final_model_prod(Final_model,Data.columns.to_list(),scaler)

Please enter valid age: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 40
Please enter valid bp: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 179
Please enter valid al: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 3
Please enter valid su: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 1
Please enter valid bgr: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 165
Please enter valid bu: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 34
Please enter valid sc: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 0.9
Please enter valid sod: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 129
Please enter valid pot: 
 Note: If this is a boolean parame

In [53]:
# Test the final model
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings("ignore", category=UserWarning)
Final_model_prod(Final_model,Data.columns.to_list(),scaler)

Please enter valid age: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 40
Please enter valid bp: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 179
Please enter valid al: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 3
Please enter valid su: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 1
Please enter valid bgr: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 165
Please enter valid bu: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 34
Please enter valid sc: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 0.9
Please enter valid sod: 
 Note: If this is a boolean parameter please provide values as 1 for 'yes' and 0 for 'No' 129
Please enter valid pot: 
 Note: If this is a boolean parame